In [1]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob
import os

In [10]:
# dirpath = r"/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/masked_shapelet002/"
# out_fp = r"/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/shapelet_Mosaic002.tif"
# dirpath = r"/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/masked_shapelet003/"
# out_fp = r"/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/shapelet_Mosaic003.tif"
# dirpath = r"/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/masked_shapelet001/"
# out_fp = r"/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/shapelet_Mosaic001.tif"
dirpath = r"/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/masked_shapelet0005/"
out_fp = r"/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/shapelet_Mosaic0005.tif"
search_criteria = "*.tif"
q = os.path.join(dirpath, search_criteria)
tif_fps = glob.glob(q)

In [11]:
src_files_to_mosaic = []
for fp in tif_fps:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)


In [4]:
src_files_to_mosaic

[<open DatasetReader name='/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/masked_shapelet001/shapelet_image_masked_1981.tif' mode='r'>,
 <open DatasetReader name='/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/masked_shapelet001/shapelet_image_masked_1860.tif' mode='r'>,
 <open DatasetReader name='/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/masked_shapelet001/shapelet_image_masked_2357.tif' mode='r'>,
 <open DatasetReader name='/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/masked_shapelet001/shapelet_image_masked_274.tif' mode='r'>,
 <open DatasetReader name='/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/masked_shapelet001/shapelet_image_masked_243.tif' mode='r'>,
 <open DatasetReader name='/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/masked_shapelet001/shapelet_image_masked_2060.tif' mode='r'>,
 <open

In [12]:
mosaic, out_trans = merge(src_files_to_mosaic,nodata=999)

In [13]:
mosaic.shape

(1, 36554, 43440)

## mosiac and write out masked shapelet images

In [14]:
out_meta = src.meta.copy()

out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                 "crs": "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
                            }
                            )

In [15]:
with rasterio.open(out_fp, "w", **out_meta) as dest:
    dest.write(mosaic)

In [16]:
for src in src_files_to_mosaic:
    src.close()

## create a binary crop expansion map 2000-2015

In [17]:
import numpy as np

In [18]:
out_binary_fp = r"/media/sitian/HDD1/BULCU_TIFFS/BULCU_grid2500_2016backward/shapelet_analysis/masked_shapelet_Mosaic_binary.tif"
mosaic_binary = np.copy(mosaic)
mosaic_binary[mosaic_binary>0] =1
mosaic_binary[mosaic_binary<0] =0
out_binary_meta = src.meta.copy()

out_binary_meta.update({"driver": "GTiff",
                 "height": mosaic_binary.shape[1],
                 "width": mosaic_binary.shape[2],
                 "transform": out_trans,
                 "crs": "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
                            }
                            )

In [19]:
with rasterio.open(out_binary_fp, "w", **out_binary_meta) as dest:
    dest.write(mosaic_binary)